In [ ]:
def graph():
    global accX_Hist, accY_Hist, accZ_Hist, threadRunning
    
    threadRunning = True
    
    while True:

        steps = find_peaks(accY_Hist, distance=50, threshold=0.4)
        clear_output(wait=True)
        print("Steps = ", len(steps[0])*2)
        #plt.ylim(-5,25)
        #plt.title(f"Steps = {len(steps[0])*2}")
        
        

        #plt.plot(accX_Hist, color='blue')
        #plt.plot(accY_Hist, color='red')
        #plt.plot(accZ_Hist, color='green')

        #plt.pause(0.005)

In [ ]:
import socket, traceback
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import threading

from scipy.signal import find_peaks


plt.rcParams["figure.figsize"] = (20,9)

%matplotlib inline

plt.rcParams["figure.figsize"] = (20,9)

host = ''
port = 5555

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
s.bind((host, port))

#used for debugging
accX_Hist = []
accY_Hist = []
accZ_Hist = []

threadRunning = False

thread = threading.Thread(target=graph)

print("Success binding")
while 1:
    # clear_output(wait=True)
    message, address = s.recvfrom(8192)
    messageString = message.decode("utf-8")
    timestamp, _, accX, accY, accZ = messageString.split(",")[:5]
    accX, accY, accZ = float(accX), float(accY), float(accZ)
    accX_Hist.append(accX)
    accX_Hist = accX_Hist[-1000:]
    accY_Hist.append(accY)
    accY_Hist = accY_Hist[-1000:]
    accZ_Hist.append(accZ)
    accZ_Hist = accZ_Hist[-1000:]
    
    
    
    if threadRunning == False:
        thread.start()
    # plt.ylim(-11,11)
    
    # plt.plot(accX_Hist, color='blue')
    # plt.plot(accY_Hist, color='red')
    # plt.plot(accZ_Hist, color='green')
    
    # plt.pause(0.005)
    

In [ ]:
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

m = 100
n = 100
matrix = np.random.normal(0, 1, size=(m, n))

fig = plt.figure()
ax = fig.add_subplot(111)

for i in range(m):
    ax.clear()
    ax.plot(matrix[i, :])
    display(fig)
    clear_output(wait=True)
    plt.pause(0.2)

In [3]:
import random

from paho.mqtt import client as mqtt_client


broker = '192.168.0.131'
port = 1883
topic = "sensor/data"
# generate client ID with pub prefix randomly
client_id = f'python-mqtt-{random.randint(0, 100)}'
# username = 'emqx'
# password = 'public'


def connect_mqtt() -> mqtt_client:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")

    client.subscribe(topic)
    client.on_message = on_message


def run():
    client = connect_mqtt()
    subscribe(client)
    client.loop_forever()


run()

ModuleNotFoundError: No module named 'paho'